In [20]:
!pwd

/home/yorshula/yorshula_master


In [1]:
from lra.layers import *
from lra.models import *
from lra.setups import *
from lra.utils  import *

import sys
sys.path.append('/home/yorshula/long-range-arena')

from lra_benchmarks.text_classification.input_pipeline import get_tc_datasets

In [2]:
train_dataset, valid_dataset, test_dataset, encoder = get_tc_datasets(1, 'imdb_reviews', batch_size=BPE_CLS_SETUP['batch_size'], max_length=BPE_CLS_SETUP['max_length'])

2022-05-02 12:38:52.393137: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 12:38:52.395647: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 12:38:52.459918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 12:38:52.462208: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 12:38:52.464264: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

INFO:tensorflow:Finished preprocessing


2022-05-02 12:39:13.943036: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
INFO:tensorflow:Finished preprocessing


INFO:tensorflow:<MapDataset element_spec={'Source': TensorSpec(shape=(), dtype=tf.string, name=None), 'Target': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


INFO:tensorflow:<MapDataset element_spec={'Source': TensorSpec(shape=(), dtype=tf.string, name=None), 'Target': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


In [7]:
BPE_CLS_SETUP['device'] = 'cuda:1'
#LISTOPS_SETUP['model_type'] = ClassificationTransformerSkip

def att_factory(hidden_dim, qkv_dim, num_heads, dropout_rate):
    
    lka = nn.Sequential(
        AMGOLU(num_heads, qkv_dim, qkv_dim // num_heads // 4, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.0),
        AMGOLU(num_heads, qkv_dim, qkv_dim // num_heads // 4, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.0),
        AMGOLU(num_heads, qkv_dim, qkv_dim // num_heads // 4, dropout_rate, nn.Sigmoid(), nn.Softplus(), False, LAMBDA=0.0),
        
        #GatedOrthoKernel(num_heads, hidden_dim, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.1),
        #GatedOrthoKernel(num_heads, hidden_dim, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.1),
        #GatedOrthoKernel(num_heads, hidden_dim, dropout_rate, nn.Sigmoid(), nn.Softplus(), False, LAMBDA=0.1)

        #HeadWiseFF(num_heads, qkv_dim, dropout_rate, nn.Softplus(), use_bias=False, LAMBDA=0.1),   
    )
    return LKAAttention(hidden_dim, qkv_dim, num_heads, dropout_rate, lka)
    
    #return FtAttention(hidden_dim, qkv_dim, num_heads, dropout_rate)
    #return SimpleAttention(hidden_dim, qkv_dim, num_heads, dropout_rate, use_lin=True)
    #return SimpleAttention(hidden_dim, qkv_dim, num_heads, dropout_rate, use_lin=False)

model, criterion, optimizer, schedule_func, scheduler = training_setup(BPE_CLS_SETUP, encoder.vocab_size, att_factory)

In [ ]:
import torch

BPE_CLS_SETUP['device'] = 'cuda:1'
#BPE_CLS_SETUP['steps'] = 20
#BPE_CLS_SETUP['eval_period'] = 10

test_accuracy = [  ]

for i in range(1): ####!!!!!!!!!!!!!!
  path = 'model_to_test_' + str(i) + '.b'

  model, criterion, optimizer, schedule_func, scheduler = training_setup(BPE_CLS_SETUP, encoder.vocab_size, att_factory)

  checkpoint = train_cls_model(BPE_CLS_SETUP, model, path, train_dataset, valid_dataset, optimizer, criterion, scheduler)
  model.load_state_dict(checkpoint['model_state_dict'])
  
  _, _, acc = cls_test(model, criterion, test_dataset, BPE_CLS_SETUP['device'])
  test_accuracy.append(acc)

test_accuracy = np.mean(test_accuracy)

print(f'\nTotal accuracy: {test_accuracy:.4f}')

In [9]:
model

ClassificationTransformer(
  (embed_layer): TEmbedding(
    (embedding): Embedding(257, 256, padding_idx=0)
  )
  (blocks): ModuleList(
    (0): TBlock(
      (layernorm_input): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
      (layernorm_inter): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
      (attention): LKAAttention(
        (q): Linear(in_features=256, out_features=256, bias=True)
        (k): Linear(in_features=256, out_features=256, bias=True)
        (v): Linear(in_features=256, out_features=256, bias=True)
        (lka): Sequential(
          (0): AMGOLU(
            (orth_weight): HWLinear()
            (gate_weight_a): HWLinear()
            (gate_weight_b): HWLinear()
            (kernel_nonlinearity): Identity()
            (gate_nonlinearity): Sigmoid()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): AMGOLU(
            (orth_weight): HWLinear()
            (gate_weight_a): HWLinear()
            (gate_weight_b): HWLine